In [1]:
import pandas as pd
import os
import re
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [23]:
import keras_vggface
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

print(keras_vggface.__version__)

import keras.utils as image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
#from keras.metrics import PearsonCorrelationCoefficient

from scipy.stats import pearsonr
from tensorflow.keras import backend as K


0.6


In [3]:
!ls

Images		     df_female.csv  eda.ipynb		  streamlit_try.py
customized_model.h5  df_male.csv    flask.py		  updated_data.csv
data.csv	     df_test.json   get_embeddings.ipynb
deploy.ipynb	     df_train.json  run_regression.ipynb


In [4]:
updated_data = pd.read_csv('updated_data.csv', index_col = None)
updated_data.head()

bmi  gender  is_training       name
0  34.207396    Male            1  img_0.bmp
1  26.453720    Male            1  img_1.bmp
2  34.967561  Female            1  img_2.bmp
3  22.044766  Female            1  img_3.bmp
4  25.845588  Female            1  img_6.bmp

### <strong>VGGFace - RESNET50 architecture</strong>

In [5]:
file_list = os.listdir('/home/jupyter/ML Final/Images')
len(file_list)

3963

In [93]:
# Load the VGGFace model with pre-trained weights
#vggface_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [10]:
# # Convert the image to a Numpy array and preprocess it for the VGGFace model
# img = load_img('/home/jupyter/ML Final/Images/img_0.bmp', target_size = (224, 224))
# img = image.img_to_array(img)
# img = np.expand_dims(img, axis = 0)
# img = preprocess_input(img, version=1)
# # Get the face embeddings
# embeddings = vggface_model.predict(img) #Convolution Features

# print(embeddings.shape)

1/1 [==============================] - 1s 1s/step
(1, 2048)


### <strong>VGGFace -VGG16 architecture</strong>

In [11]:
# layer_name = 'fc6' # Specific Layer Features
# vgg_model = VGGFace(model='vgg16', include_top=True, input_shape=(224, 224, 3), pooling='avg')
# out = vgg_model.get_layer(layer_name).output
# vgg_model_new = Model(vgg_model.input, out)
# embeddings = vgg_model_new.predict(img)
# print(embeddings.shape)

1/1 [==============================] - 0s 276ms/step
(1, 4096)


##### <strong><em>Get encoddings</em></strong>

In [6]:
image_path = '/home/jupyter/ML Final/Images/'

def get_face_encoding(image_name, model):

    img = image.load_img(image_path + image_name, target_size = (224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = preprocess_input(img, version=1)
    
    embed = model.predict(img, verbose = 0) 
    return embed

In [7]:
vggface_resnet = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
face_encodings_vggresnet = []

for images in updated_data.name:
    face_enc = get_face_encoding(images, vggface_resnet)
    face_encodings_vggresnet.append(face_enc)

2023-05-17 02:54:31.230408: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-17 02:54:31.230463: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-17 02:54:31.230497: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-11): /proc/driver/nvidia/version does not exist
2023-05-17 02:54:31.230890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the 

In [8]:
layer_name = 'fc6' 
vgg_model = VGGFace(model='vgg16', include_top=True, input_shape=(224, 224, 3), pooling='avg')
out = vgg_model.get_layer(layer_name).output
vggface_vggfc6 = Model(vgg_model.input, out)

face_encodings_vggfc6 = []

for images in updated_data.name:
    face_enc = get_face_encoding(images, vggface_vggfc6)
    face_encodings_vggfc6.append(face_enc)

In [9]:
updated_data['face_encodings_vggfc6'] = face_encodings_vggfc6
updated_data['face_encodings_vggresnet'] = face_encodings_vggresnet


In [10]:
updated_data.head()

bmi  gender  is_training       name  \
0  34.207396    Male            1  img_0.bmp   
1  26.453720    Male            1  img_1.bmp   
2  34.967561  Female            1  img_2.bmp   
3  22.044766  Female            1  img_3.bmp   
4  25.845588  Female            1  img_6.bmp   

                               face_encodings_vggfc6  \
0  [[-1.4681633, -0.5275203, -22.067028, -0.86678...   
1  [[-0.20396894, -0.7145279, -62.69967, -0.70725...   
2  [[-0.87429816, -0.51126647, -30.323956, -0.587...   
3  [[-0.71330357, -0.47140464, -33.486244, -0.426...   
4  [[0.9988322, -0.3236777, -21.522282, -0.485504...   

                            face_encodings_vggresnet  
0  [[0.073260844, 0.10817158, 0.7255822, 0.0, 0.0...  
1  [[0.28265408, 1.3589354, 0.0, 0.67086667, 0.0,...  
2  [[0.012803124, 6.673352, 0.0, 0.1777949, 4.118...  
3  [[0.11812438, 2.272212, 4.2830234, 0.0, 0.0355...  
4  [[0.77834415, 0.0, 0.25624114, 0.034757275, 1....

In [28]:

json_str = updated_data.to_json(orient='records')

with open('updated_data_encoded.json', 'w') as f:
    f.write(json_str)

##### <strong><em>Store encoddings</em></strong>

In [16]:
df_train = updated_data[updated_data['is_training'] == 1].reset_index(drop = True)
df_test = updated_data[updated_data['is_training'] == 0].reset_index(drop = True)

In [32]:
#df_test.head()

bmi  gender  is_training          name  \
0  29.698495    Male            0  img_3369.bmp   
1  30.845918  Female            0  img_3370.bmp   
2  24.389796  Female            0  img_3371.bmp   
3  36.258679    Male            0  img_3372.bmp   
4  27.891291    Male            0  img_3373.bmp   

                               face_encodings_vggfc6  \
0  [[-3.9410083, -0.6707837, -53.463085, -0.90942...   
1  [[-0.16348448, -0.55346894, -22.074692, -0.896...   
2  [[1.892206, -0.81442004, -27.296654, -0.596103...   
3  [[-1.9660703, -0.30473545, -4.8362894, -0.7448...   
4  [[-1.2365557, -0.18508132, 9.221899, -0.271934...   

                            face_encodings_vggresnet  
0  [[0.016919836, 1.3312423, 0.0, 0.0, 0.0, 2.750...  
1  [[0.7402006, 5.383691, 0.31813782, 0.030300671...  
2  [[0.0030249187, 5.466592, 0.0, 0.6811399, 0.0,...  
3  [[0.0, 0.531619, 0.0, 0.0, 0.0, 2.2228844, 0.1...  
4  [[0.0, 0.0, 1.0690471, 12.230001, 0.0, 4.27751...

In [31]:
len(updated_data.face_encodings_vggresnet[0][0])

2048

In [17]:
json_str = df_train.to_json(orient='records')

with open('df_train.json', 'w') as f:
    f.write(json_str)

In [18]:
json_str = df_test.to_json(orient='records')

with open('df_test.json', 'w') as f:
    f.write(json_str)

### <strong>Finetuning</strong>

In [19]:
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

for layer in base_model.layers:
    layer.trainable = False
    
#base_model.summary()

In [20]:
import tensorflow_probability as tfp
def pearson_corr(y_true, y_pred):
   
    corr = tfp.stats.correlation(y_true,y_pred) 
    return corr

tf.keras.utils.get_custom_objects()['pearson_corr'] = pearson_corr

In [ ]:
# Get the output of the last layer of the ResNet50 model
x = base_model.output

# Flatten the output of the last layer to create a 1D vector
x = Flatten()(x)

# Add a dense layer with 128 neurons and 'relu' activation function
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# x = Dense(64, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.2)(x)

# Add the final dense layer with one neuron and a linear activation function
out = Dense(1, activation='linear')(x)

# Create a new model that includes the ResNet50 model and the new layers
model = Model(base_model.input, out)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[pearson_corr])


#model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]']           
 ation)                         )                                                           

In [56]:
X_train_images =  np.zeros((len(df_train), 224, 224, 3))
for i, filename in enumerate(df_train['name']):
    img = load_img('/home/jupyter/ML Final/Images/'+filename, target_size = (224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = preprocess_input(img, version=2)
    #image = Image.open(path + filename)
    #image = image.resize((224, 224))
    X_train_images[i] = np.array(img)
    
X_test_images =  np.zeros((len(df_test), 224, 224, 3))
for i, filename in enumerate(df_test['name']):
    img = load_img('/home/jupyter/ML Final/Images/'+filename, target_size = (224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = preprocess_input(img, version=2)
    #image = Image.open(path + filename)
    #image = image.resize((224, 224))
    X_test_images[i] = np.array(img)

In [57]:
y_train_bmi = np.array(df_train['bmi'])
y_test_bmi = np.array(df_test['bmi'])

In [58]:
print(X_train_images.shape)
print(X_test_images.shape)
print(y_train_bmi.shape)
print(y_test_bmi.shape)

(3210, 224, 224, 3)
(752, 224, 224, 3)
(3210,)
(752,)


In [59]:
history = model.fit(X_train_images, y_train_bmi, batch_size=32, epochs=3, validation_data=(X_test_images, y_test_bmi))

Epoch 1/3
101/101 [==============================] - 84s 797ms/step - loss: 1101.7786 - pearson_corr: 0.4279 - val_loss: 1112.4683 - val_pearson_corr: 0.6220
Epoch 2/3
101/101 [==============================] - 78s 769ms/step - loss: 1078.7247 - pearson_corr: 0.6516 - val_loss: 1128.9197 - val_pearson_corr: 0.6613
Epoch 3/3
101/101 [==============================] - 77s 767ms/step - loss: 1063.3429 - pearson_corr: 0.6773 - val_loss: 1119.9336 - val_pearson_corr: 0.6725


In [71]:
y_pred = model.predict(X_test_images)

corr, _ = pearsonr(y_test_bmi, y_pred)
print("Test correlation coefficient:", corr)


24/24 [==============================] - 15s 620ms/step
Test correlation coefficient: [0.6403926799343017]


In [85]:
model.save('customized_model.h5')

In [69]:
import tensorflow as tf

In [76]:
y_test_bmi = tf.reshape(y_test_bmi, shape=(-1,))
y_pred = tf.reshape(y_pred, shape=(-1,))
tfp.stats.correlation(y_test_bmi,y_pred, event_axis=-1) 

ValueError: sample_axis ((0,)) and event_axis ((0,)) overlapped

In [78]:
y_pred

<tf.Tensor: shape=(752,), dtype=float32, numpy=
array([ 1.09586945e+01,  1.92701556e-02, -7.58647394e+00,  3.64185786e+00,
        2.17600897e-01,  5.33654070e+00, -4.19941235e+00,  1.28894863e+01,
        8.80793953e+00,  8.21286082e-01, -7.60042906e+00, -3.10863376e+00,
       -5.76316309e+00, -2.90116024e+00, -6.42994261e+00,  5.53316784e+00,
        3.52431655e-01,  4.15644789e+00,  2.20982265e+00, -3.10074925e+00,
        1.00024443e+01,  4.78847408e+00, -2.12772560e+00,  1.00037107e+01,
        8.50867271e+00, -1.46765053e-01, -1.20047522e+00, -1.10513854e+00,
        1.86181545e+00,  6.51499319e+00, -7.71720886e+00,  6.06511736e+00,
       -4.70504665e+00, -2.40529418e+00, -3.87799931e+00, -4.00857735e+00,
       -7.08114672e+00,  1.30570316e+01,  1.06522179e+00, -1.40945673e+00,
       -5.17215395e+00,  4.81869459e+00, -5.43441963e+00,  3.90988064e+00,
       -4.21265185e-01,  3.71391058e+00, -1.73768592e+00,  2.83162451e+00,
       -2.12177086e+00,  4.48492432e+00, -3.12019134